In [14]:
import os, json
from datetime import datetime,date
from notion_client import Client, AsyncClient
from pprint import pprint

In [3]:
notionIntegration="secret_HORF73Hvw3Cg7kAH9ZvRGY6FaJ0GulrsuZa0WvJ2ObN"
notion = Client(auth=notionIntegration)

In [4]:
#List Users
list_users_response = notion.users.list()
pprint(list_users_response)

{'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'results': [{'avatar_url': None,
              'id': '05d0ee69-2290-4d4d-b357-ae576397a133',
              'name': 'Prinjal',
              'object': 'user',
              'person': {'email': 'dave.pri@northeastern.edu'},
              'type': 'person'},
             {'avatar_url': None,
              'bot': {'owner': {'type': 'workspace', 'workspace': True},
                      'workspace_name': "Prinjal's Notion"},
              'id': '685c4c01-916c-40ef-a9cf-4e80887ad68c',
              'name': 'Prinjal Dave',
              'object': 'user',
              'type': 'bot'}],
 'type': 'user',
 'user': {}}


In [5]:
for x in notion.search()['results']:
    if(x["object"]=="page"):
        pprint(x["properties"]["Due Date"])

{'date': {'end': '2022-12-26', 'start': '2022-12-23', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': '2022-12-26', 'start': '2022-12-21', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': None, 'start': '2023-01-10', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': None, 'start': '2022-12-25', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': None, 'start': '2022-12-24', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': '2022-12-21', 'start': '2022-12-18', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': '2022-12-26', 'start': '2022-12-23', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': {'end': '2022-12-26', 'start': '2022-12-23', 'time_zone': None},
 'id': 'z.OF',
 'type': 'date'}
{'date': None, 'id': 'z.OF', 'type': 'date'}
{'date': None, 'id': 'z.OF', 'type': 'date'}


In [18]:
for x in notion.search()['results']:
    if(x["object"] == "page"):
        statusName = x["properties"]["Status"]["status"]["name"]
        dueDateObj = x["properties"]["Due Date"]["date"]
        name = x["properties"]["Name"]["title"][0]["text"]["content"]
        pageId = x["id"]
        if(dueDateObj != None):
            start = dueDateObj["start"]
            end = dueDateObj["end"]
            if(start != None):
                start = datetime.strptime(dueDateObj["start"], "%Y-%m-%d").date()
            else:
                start = None
            if(end != None):
                end = datetime.strptime(dueDateObj["end"], "%Y-%m-%d").date()
            else:
                end = None
            currentDate = date.today()

            if(end == None):
                    if(start < currentDate and (statusName == "Next Up" or statusName == "In Progress")):
                        res = updatePageHelper(
                            pageId=pageId, statusName="Reschedule", dueDateStart=start, dueDateEnd=end)
                    elif(start >= currentDate and (statusName == "Next Up")):
                        res = updatePageHelper(
                            pageId=pageId, statusName="In Progress", dueDateStart=start, dueDateEnd=end)
            elif(end != None):
                if(start <= currentDate and end <currentDate and (statusName=="In Progress" or statusName=="Next Up")):
                    res = updatePageHelper(pageId=pageId, statusName="Reschedule", dueDateStart=start,dueDateEnd=end)
                elif(start <= currentDate and end >currentDate and (statusName=="Next Up")):
                    res = updatePageHelper(pageId=pageId, statusName="In Progress", dueDateStart=start,dueDateEnd=end)
                elif(start>currentDate and (statusName=="In Progress")):
                    res = updatePageHelper(pageId=pageId, statusName="Next Up", dueDateStart=start,dueDateEnd=end)



In [10]:
def updatePageHelper(pageId,statusName, dueDateStart, dueDateEnd):
    if(dueDateStart!=None):
        dueDateStart=datetime.strftime(dueDateStart,"%Y-%m-%d")
    if(dueDateEnd!=None):
        dueDateEnd=datetime.strftime(dueDateEnd,"%Y-%m-%d")
    res=notion.pages.update(page_id=pageId,
                   properties={"Status":{
                       "status":{
                           "name":statusName
                       }
                   },"Due Date":{
                       "date":{
                           "start":dueDateStart,
                           "end":dueDateEnd}
                   }})
    return res